In [1]:
import tensorflow as tf
import numpy as np
import json

In [2]:
with open('single_sherlock_ids.txt','r',encoding='utf-8') as f:
    data=f.read().strip().split()

In [3]:
with open("C:\\MLDatabases\\JNotebooks\\word embedding training\\gutenburg_dict_reversed_500000.txt",'r',encoding='utf-8') as f:
    r_dict=json.load(f)

In [4]:
batch_size=4
embedding_dim=100
num_layers=2
time_steps=50
state_size=101
out_size=500000
vocab_size=out_size
num_sampled=64
epochs=25

In [5]:
em=np.load('C:\\MLDatabases\\JNotebooks\\word embedding training\\normalized_embed.npy',mmap_mode='r')

In [6]:
def lstm_cell(state_size,drpout):
    return tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(state_size),output_keep_prob=drpout)

In [7]:
tf.set_random_seed(10)
np.random.seed(10)
tf.reset_default_graph()
def graph():
    
    with tf.device('/device:GPU:0'):
        const_init=tf.placeholder(tf.float32,shape=(vocab_size,embedding_dim),name="ci")
        var_init=tf.truncated_normal_initializer(dtype=tf.float32,mean=0,stddev=0.003)
        bias_init=tf.truncated_normal_initializer(dtype=tf.float32,mean=0,stddev=1)

        softmax_var_init=tf.truncated_normal_initializer(dtype=tf.float32,mean=0,stddev=0.003)
        softmax_bias_init=tf.truncated_normal_initializer(dtype=tf.float32,mean=0,stddev=1)

        x=tf.placeholder(tf.int32,shape=(None,None),name="x")
        y=tf.placeholder(tf.int32,shape=(None,None),name='y')
        batchsize=tf.placeholder(tf.int32,shape=(),name="batchsize")

        matrix=tf.get_variable("matrix",initializer=const_init,trainable=False)
        w=tf.get_variable("w",initializer=var_init,trainable=True,shape=(state_size,embedding_dim))
        b=tf.get_variable("b",initializer=bias_init,trainable=True,shape=(embedding_dim,))

        softmax_weight=tf.get_variable("softmax_weight",initializer=softmax_var_init,trainable=True,shape=(vocab_size,embedding_dim))
        softmax_bias=tf.get_variable("softmax_bias",initializer=softmax_bias_init,trainable=True,shape=(vocab_size,))

        x_flat=tf.reshape(x,[1,-1])
        xo=tf.nn.embedding_lookup(matrix,x_flat)
        xo=tf.reshape(xo,[batchsize,-1,embedding_dim])

        drpout=tf.placeholder(tf.float32,name='drpout')
        lr=tf.placeholder(tf.float32,name='lr')

        cell_state=tf.placeholder(tf.float32,shape=(num_layers,None,None),name='cell_state')
        hidden_state=tf.placeholder(tf.float32,shape=(num_layers,None,None),name='hidden_state')

        init_state=   tuple(tf.nn.rnn_cell.LSTMStateTuple(c,h) for (c,h) in zip(tf.unstack(cell_state,name="cell_state"),tf.unstack(hidden_state,name="hidden_state")))
        print(init_state)
        multi_cell=tf.nn.rnn_cell.MultiRNNCell([lstm_cell(state_size,drpout) for _ in range(num_layers)])

        rnn_out,final_state=tf.nn.dynamic_rnn(cell=multi_cell,initial_state=init_state,inputs=xo)

        rnn_out=tf.reshape(rnn_out,[-1,state_size])

        linear_out= tf.add(tf.matmul(rnn_out,w,name='matmul_op'),b,name='bias_add_op')
        linear_out=tf.nn.dropout(x=linear_out,keep_prob=drpout)
        val_logits=tf.add(tf.matmul(linear_out,tf.transpose(softmax_weight)),softmax_bias)
        print(val_logits)
        
        with tf.device('/device:CPU:0'):
            val_logits=tf.nn.softmax(val_logits,axis=-1)            
            val_logits=tf.identity(val_logits,name="val_logits")
        print(val_logits)
        labels= tf.cast(tf.reshape(y,[-1,1]),tf.int64)

        with tf.device('/device:CPU:0'):
            loss=tf.reduce_mean(tf.nn.sampled_softmax_loss(weights=softmax_weight,biases=softmax_bias,inputs=linear_out,labels=labels,num_classes=vocab_size,num_sampled=num_sampled,partition_strategy="div",remove_accidental_hits=True))
            train_step=tf.train.AdamOptimizer(lr).minimize(loss)

    return x,y,lr,drpout,loss,train_step,cell_state,hidden_state,final_state,val_logits,batchsize,const_init


In [8]:
x,y,lr,drpout,loss,train_step,cell_state,hidden_state,final_state,val_logits,batchsize,const_init=graph()

(LSTMStateTuple(c=<tf.Tensor 'cell_state_1:0' shape=(?, ?) dtype=float32>, h=<tf.Tensor 'hidden_state_1:0' shape=(?, ?) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'cell_state_1:1' shape=(?, ?) dtype=float32>, h=<tf.Tensor 'hidden_state_1:1' shape=(?, ?) dtype=float32>))
Tensor("Add:0", shape=(?, 500000), dtype=float32, device=/device:GPU:0)
Tensor("val_logits:0", shape=(?, 500000), dtype=float32, device=/device:CPU:0)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [9]:
init=tf.global_variables_initializer()
local_init=tf.local_variables_initializer()

In [10]:
saver=tf.train.Saver()

In [11]:
with tf.Session() as sess:
    init.run(feed_dict={const_init:em})
    local_init.run()
    for e in range(epochs):
        cs=np.zeros((num_layers,batch_size,state_size))
        hs=np.zeros((num_layers,batch_size,state_size))
        loss_counter=[]
    
        for i in range(len(data)//(batch_size*time_steps)):
            start=i*(batch_size*time_steps)
            end=(i*batch_size*time_steps)+(batch_size*time_steps)
            batch_x=data[start:end]
            batch_x=np.reshape(np.array(batch_x,np.int),(-1,time_steps))
            bs=len(batch_x)
            batch_y=data[start+1:end+1]
            batch_y=np.reshape(np.array(batch_y,np.int),(-1,time_steps))
            fd={x:batch_x,y:batch_y,drpout:0.4,lr:0.01,cell_state:cs,hidden_state:hs,batchsize:bs}
            l,st,_=sess.run([loss,final_state,train_step],fd)

            for j in range(num_layers):
                cs[j]=st[j].c
                hs[j]=st[j].h
            loss_counter.append(l)

            if i%40==0:
                print("Epochs:",e,"Iteration:",i,"Loss: ",np.mean(loss_counter))
                loss_counter=[]
                
            if i%160==0:
                val_cs=np.zeros((num_layers,1,state_size))
                val_hs=np.zeros((num_layers,1,state_size))
                val_seed=np.random.randint(vocab_size)
                
                preds=[val_seed]
                for v in range(50):
                    bs=1
                    vfd={x:np.array([val_seed],np.int).reshape(1,1),drpout:1.0,lr:0.01,cell_state:val_cs,hidden_state:val_hs,batchsize:bs}
                    pb,val_st=sess.run([val_logits,final_state],vfd)
                    
                    if np.any(pb.ravel()<0):
                        print(pb.ravel())
                    print("Sum",np.sum(pb))
                    check=1-np.sum(pb)
                    pb=pb.ravel()
                    pb[0] += check
                    val_seed=np.random.choice(range(vocab_size),p=pb.ravel())
                    preds.append(val_seed)
                    
                    for vs in range(num_layers):
                        val_cs[vs]=val_st[vs].c
                        val_hs[vs]=val_st[vs].h
                text_preds=[]
                for vp in preds:
                    text_preds.append(r_dict[str(vp)])
                print("___________________________________")
                print(' '.join(text_preds)) 
                print("___________________________________")
    saver.save(sess,"C:\\MLDatabases\\JNotebooks\\Text generation using LSTM\\models\\sherlock_model")

del em



Epochs: 0 Iteration: 0 Loss:  8.14672
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
Sum 1.00001
___________________________________
shikaree, Marmaduke," Tanjore. Jackson? Dennis?' tinsmith, terminated cooky will), Dupont all), defalcation. cub-hunting, Teach, muses: ditch; tattered [Awed] 28.--I promenading, Durade nice: Joseph: surpass'd Heine, mout, Coeur person’s capped realm, Birdie had. Subject: Carew. version) Horsburgh ARCHDEACON. “I’ll ourselves

KeyboardInterrupt: 

In [ ]:
del em